In [1]:
import os
try:
    os.chdir(base_dir) #if the start up file has already been run, this will change the dir back to base
except NameError:
    base_dir = None #if the start up file hasn't been run yet, this will allow us to run this cell without crashing

In [2]:
%run ./startup_file.ipynb #run startup file and install libraries if necessary

# GRIDDING

## Author: haley synan

##### *Last run on 2024-07-24*

> *History:* <br>
>> * 6/28/2024: originally written <br>
### Objectives: 
* Re-grid data to 5 km (same spatial resolution as current Seascapes product)
* Re-grid data to degree equivalent to 5 km

### TWO METHODS
There are two methods used for regridding data in this notebook. The first uses the **xarray** library, and the second uses the **geopandas** library.

## XARRAY METHOD
The xarray method is quite simple. It uses a function in the xarray library, called [*regrid*](https://pypi.org/project/xarray-regrid/). It takes two arguments, the source grid (the data that will be regridded) and the target grid (the GOAL grid size). The target grid can be in degrees, km, etc. 

#### LOAD SOURCE DATA

For this example, the source data (that we want to regrid) is [ASCPO Daily SST data](https://coastwatch.noaa.gov/erddap/griddap/noaacwLEOACSPOSSTL3SCDaily.html). It is CURRENTLY gridded to 0.02 degrees.

We are going to read and download the data, but if you already have it downloaded, you can use load it in using xarray. 

CD to correct data directory

In [3]:
os.chdir(data_dir) #cd to data directory 
#create new project data folder 
proj_data = '/SST' #name of project
data_dir_fold = data_dir+proj_data #path for new folder 
isexist = os.path.exists(data_dir_fold) #check if path exists 
#print(isexist)
if str(isexist) == 'False': #if path doesn't exist already, make it
    os.mkdir(data_dir_fold)

In [4]:
start_date = '2021-06-01' #(yyyy-mm-dd format)
end_date = '2021-08-31'
url = ''.join(['https://coastwatch.noaa.gov/erddap/griddap/noaacwLEOACSPOSSTL3SCDaily.nc?sea_surface_temperature%5B('+start_date+'T12:00:00Z):1:('+end_date+'T12:00:00Z)%5D%5B(34.40918):1:(46.362305)%5D%5B(-63.585942):1:(-77.681645)%5D'
               ]) 

def url2date(url, nu): #write function to grab the start and end dates of the data inquiry to use them for naming our data file
    dat = url.split('(')
    s_dat = [s for s in dat if nu in s]
    s_dat = s_dat[0].split('T')
    s_dat = s_dat[0].split('-')
    s_dat = s_dat[0]+s_dat[1]+s_dat[2]
   
    return(s_dat)
 
s_date = url2date(url,nu=start_date) # for start date: nu = 1
e_date = url2date(url,nu=end_date) #for end date: nu = 2   
fname = "/DD1_" + s_date + '_'+ e_date + ".nc" #create unique filename 
#follow kims naming structure 
#DD8_yyyymmdd_yyyymmdd start end dates #write function to get that parts of URL 
file = data_dir_fold+fname
urllib.request.urlretrieve(url, file) #download data

('C:/users/haley.synan/Documents/SEASCAPES/DATA/SST/DD1_20210601_20210831.nc',
 <http.client.HTTPMessage at 0x19afe344bd0>)

#### Open data 
Since this is our "source" data, we are going to name it ds_source

In [5]:
ds_source = xr.open_dataset(file, decode_cf=True) #open nc file 
ds_source #inspect xarray dataset

<xarray.Dataset> Size: 156MB
Dimensions:                  (time: 92, latitude: 599, longitude: 706)
Coordinates:
  * time                     (time) datetime64[ns] 736B 2021-06-01T12:00:00 ....
  * latitude                 (latitude) float32 2kB 46.37 46.35 ... 34.43 34.41
  * longitude                (longitude) float32 3kB -77.69 -77.67 ... -63.59
Data variables:
    sea_surface_temperature  (time, latitude, longitude) float32 156MB ...
Attributes: (12/63)
    acknowledgement:                        Please acknowledge the use of the...
    aggregator_version:                     V1.00
    cdm_data_type:                          Grid
    col_count:                              18000
    col_start:                              0
    collation_version:                      2.11.0
    ...                                     ...
    summary:                                Sea surface temperature retrieval...
    testOutOfDate:                          now-95days
    time_coverage_end:                      2021-08-31T12:00:00Z
    time_coverage_start:                    2021-06-01T12:00:00Z
    title:                                  Sea-Surface Temperature, NOAA ACS...
    Westernmost_Easting:                    -77.69

#### Load target data 

In [6]:
proj_data = '/SEASCAPES' #name of project
data_dir_fold = data_dir+proj_data #path for new folder 
os.chdir(data_dir_fold) #go to project data folder
os.getcwd() #check if in the correct folder

url = ''.join(['https://cwcgom.aoml.noaa.gov/erddap/griddap/noaa_aoml_seascapes_8day.nc?',
               'CLASS',
               '%5B(' + start_date+'T12:00:00Z):1:('+end_date+'T12:00:00Z)%5D',
               '%5B(46.362305):1:(34.40918)%5D%5B(-63.585942):1:(-77.681645)%5D,P%5B('+start_date+'T12:00:00Z):1:('+end_date+'T12:00:00Z)%5D%5B(46.362305):1:(34.40918)%5D%5B(-63.585942):1:(-77.681645)%5D'
               ]) 

s_date = url2date(url,nu=start_date) # for start date: nu = 1
e_date = url2date(url,nu=end_date) #for end date: nu = 2   
fname = "/DD8_" + s_date + '_'+ e_date + ".nc" #create unique filename 
#follow kims naming structure 
#DD8_yyyymmdd_yyyymmdd start end dates #write function to get that parts of URL 
file = data_dir_fold+fname
urllib.request.urlretrieve(url, file) #download data

('C:/users/haley.synan/Documents/SEASCAPES/DATA/SEASCAPES/DD8_20210601_20210831.nc',
 <http.client.HTTPMessage at 0x19aff4d6bd0>)

In [7]:
ds_target = xr.open_dataset(file, decode_cf=True) #open nc file 
ds_target #inspect xarray dataset

<xarray.Dataset> Size: 7MB
Dimensions:    (time: 12, latitude: 240, longitude: 283)
Coordinates:
  * time       (time) datetime64[ns] 96B 2021-06-02T12:00:00 ... 2021-08-29T1...
  * latitude   (latitude) float32 960B 34.42 34.47 34.53 ... 46.28 46.33 46.38
  * longitude  (longitude) float32 1kB -77.68 -77.62 -77.57 ... -63.62 -63.58
Data variables:
    CLASS      (time, latitude, longitude) float32 3MB ...
    P          (time, latitude, longitude) float32 3MB ...
Attributes: (12/38)
    acknowledgement:            The U.S. MBON projects are funded under the N...
    cdm_data_type:              Grid
    contact:                    Joaquin.Trinanes@noaa.gov/mkavanau@ceoas.oreg...
    Conventions:                COARDS, CF-1.4, Unidata Dataset Discovery v1.0
    creator_email:              Joaquin.Trinanes@noaa.gov/mkavanau@ceoas.oreg...
    creator_type:               group
    ...                         ...
    summary:                    Biogeographic framework. Space and time class...
    time_coverage_duration:     P0Y1M0DT0H0M0S
    time_coverage_end:          2021-08-29T12:00:00Z
    time_coverage_start:        2021-06-02T12:00:00Z
    title:                      8_Day Global Seascapes
    Westernmost_Easting:        -77.675

In [ ]:
import xarray_regrid
ds = ds_source.regrid.linear(ds_target) #regrid data 
#now you can use ds like normal

In [ ]:
ds #REGRIDDED DATA (matches seascapes grid)

## GEOPANDAS METHOD: POINT DATA 
The geopandas method, we create a grid and regrid the source data to that grid. The main differences is the the grid is in *degrees* and this is really only useful for point data

In [38]:
#df = ds_source.to_dataframe().reset_index() #convert xarray dataset to pandas dataframe

#import POINT DATA (csv file) 
os.chdir(data_dir)
df = pd.read_csv('ecomonchla_formatted.csv') #read validation data
df.head() #view dataframe header

,lat,lon,datetime,depth,temp,sal,chla,CT,SA,spiciness0,mlp,monthday,date,month,TEMP_NCEI_clima,SAL_NCEI_clima,sal_anom,temp_anom,ze
0,41.0183,-71.0817,2009-11-03T17:36:00Z,3,13.18,32.122,3.41,13.233622,32.274322,-0.228526,NaN,NaN,20091103,11,16.299,33.275,-1.153,-3.119,22.711675
1,41.0183,-71.0817,2009-11-03T17:36:00Z,4,13.17,32.121,2.61,13.223472,32.273317,-0.231334,NaN,NaN,20091103,11,16.299,33.275,-1.154,-3.129,25.128467
2,41.0183,-71.0817,2009-11-03T17:36:00Z,5,13.16,32.123,2.73,13.213255,32.275325,-0.231941,NaN,NaN,20091103,11,16.299,33.275,-1.152,-3.139,24.704854
3,41.0183,-71.0817,2009-11-03T17:36:00Z,6,13.19,32.121,2.78,13.243278,32.273315,-0.227295,NaN,NaN,20091103,11,16.299,33.275,-1.154,-3.109,24.535849
4,41.0183,-71.0817,2009-11-03T17:36:00Z,7,13.20,32.120,2.65,13.253203,32.272310,-0.226007,NaN,NaN,20091103,11,16.299,33.275,-1.155,-3.099,24.984329


In [39]:
import geopandas
gdf = geopandas.GeoDataFrame(df, 
            geometry=geopandas.points_from_xy(df.lon, df.lat),
            crs="EPSG:4326") #"+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")
gdf.head()

,lat,lon,datetime,depth,temp,sal,chla,CT,SA,spiciness0,mlp,monthday,date,month,TEMP_NCEI_clima,SAL_NCEI_clima,sal_anom,temp_anom,ze,geometry
0,41.0183,-71.0817,2009-11-03T17:36:00Z,3,13.18,32.122,3.41,13.233622,32.274322,-0.228526,NaN,NaN,20091103,11,16.299,33.275,-1.153,-3.119,22.711675,POINT (-71.08170 41.01830)
1,41.0183,-71.0817,2009-11-03T17:36:00Z,4,13.17,32.121,2.61,13.223472,32.273317,-0.231334,NaN,NaN,20091103,11,16.299,33.275,-1.154,-3.129,25.128467,POINT (-71.08170 41.01830)
2,41.0183,-71.0817,2009-11-03T17:36:00Z,5,13.16,32.123,2.73,13.213255,32.275325,-0.231941,NaN,NaN,20091103,11,16.299,33.275,-1.152,-3.139,24.704854,POINT (-71.08170 41.01830)
3,41.0183,-71.0817,2009-11-03T17:36:00Z,6,13.19,32.121,2.78,13.243278,32.273315,-0.227295,NaN,NaN,20091103,11,16.299,33.275,-1.154,-3.109,24.535849,POINT (-71.08170 41.01830)
4,41.0183,-71.0817,2009-11-03T17:36:00Z,7,13.20,32.120,2.65,13.253203,32.272310,-0.226007,NaN,NaN,20091103,11,16.299,33.275,-1.155,-3.099,24.984329,POINT (-71.08170 41.01830)


In [40]:
import shapely
# total area for the grid
xmin = 34
xmax = 44
ymin = -77
ymax = -63
#ymin, xmin, ymax, xmax= gdf.total_bounds
# how many cells across and down
#n_cells=30
cell_size =0.05 #(xmax-xmin)/n_cells
# projection of the grid
crs="EPSG:4326"#"+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
# create the cells in a loop
grid_cells = []
for x0 in np.arange(ymin, ymax+cell_size, cell_size ):
    for y0 in np.arange(xmin, xmax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
cell = geopandas.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs=crs)

#cell.set_crs('epsg:4326', allow_override=True)

In [41]:
cell

,geometry
0,"POLYGON ((-77.05000 34.00000, -77.05000 34.050..."
1,"POLYGON ((-77.05000 34.05000, -77.05000 34.100..."
2,"POLYGON ((-77.05000 34.10000, -77.05000 34.150..."
3,"POLYGON ((-77.05000 34.15000, -77.05000 34.200..."
4,"POLYGON ((-77.05000 34.20000, -77.05000 34.250..."
...,...
56476,"POLYGON ((-63.05000 43.80000, -63.05000 43.850..."
56477,"POLYGON ((-63.05000 43.85000, -63.05000 43.900..."
56478,"POLYGON ((-63.05000 43.90000, -63.05000 43.950..."
56479,"POLYGON ((-63.05000 43.95000, -63.05000 44.000..."


In [ ]:
ax = gdf.plot(markersize=.1, figsize=(12, 8), column='chla', cmap='jet')
plt.autoscale(False)
cell.plot(ax=ax, facecolor="none", edgecolor='grey')
ax.axis("off")

In [ ]:
merged = geopandas.sjoin(gdf, cell, how='left', op='within')
# make a simple count variable that we can sum
merged['chla']=1
# Compute stats per grid cell -- aggregate fires to grid cells with dissolve
dissolve = merged.dissolve(by="index_right", aggfunc="count")
# put this into cell
cell.loc[dissolve.index, 'chla'] = dissolve.chla.values

In [ ]:
ax = cell.plot(column='chla', figsize=(12, 8), cmap='viridis', edgecolor="grey")
plt.autoscale(False)
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.to_crs(cell.crs).plot(ax=ax, color='none', edgecolor='black')
#ax.axis('off')

## XESMF

In [8]:
import cf_xarray as cfxr
import matplotlib.pyplot as plt
import shapely
import xarray as xr
import xesmf as xe
from clisops.core.subset import subset_bbox  # For subsetting
from xclim.testing import open_dataset  

In [18]:
regridder = xe.Regridder(ds_source, ds_target, "conservative")
regridder

xESMF Regridder 
Regridding algorithm:       conservative 
Weight filename:            conservative_599x706_240x283.nc 
Reuse pre-computed weights? False 
Input grid shape:           (599, 706) 
Output grid shape:          (240, 283) 
Periodic in longitude?      False

In [ ]:
regridder(ds_source.sea_surface_temperature)

In [31]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(599,)),
        "lon": (["lon"], np.arange(706,)),
    }
)

regridder = xe.Regridder(ds_source, ds_out, "bilinear")
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_599x706_599x706.nc 
Reuse pre-computed weights? False 
Input grid shape:           (599, 706) 
Output grid shape:          (599, 706) 
Periodic in longitude?      False

In [ ]:
ds_out = regridder(ds_source)
ds_out

In [ ]:
#move this up
source_resolution =(ds_source['latitude'].max() - ds_source['latitude'].min())/(ds_source['latitude'].count()-1.) #find spatial resolution
source_resolution_lon = (ds_source['longitude'].max() - ds_source['longitude'].min())/(ds_source['longitude'].count()-1.)
print('The spatial resolution of the source dataset is' + ' '+str(source_resolution.values) +' ' + 'by' + ' '+ str(source_resolution_lon.values) + ' '+ 'degrees')
target_resolution_lat =(ds_target['latitude'].max() - ds_target['latitude'].min())/(ds_target['latitude'].count()-1.) #find spatial resolution
target_resolution_lon = (ds_target['longitude'].max() - ds_target['longitude'].min())/(ds_target['longitude'].count()-1.)
print('The spatial resolution of the target dataset is' + ' '+str(target_resolution_lat.values) +' ' + 'by' + ' '+ str(target_resolution_lon.values) + ' '+ 'degrees')

In [ ]:
xmax = max(ds_target.latitude.values)
xmin = min(ds_target.latitude.values)
ymin = min(ds_target.longitude.values)
ymax = max(ds_target.longitude.values)
ds_ot = xe.util.grid_2d(ymin, ymax, target_resolution_lon, xmin, xmax, target_resolution_lat)

In [ ]:
ds_source.squeeze(dim='latitude')

In [34]:

regridder = xe.Regridder(ds, ds_out, 'bilinear', periodic=True)

In [36]:
ds_out

<xarray.Dataset> Size: 5kB
Dimensions:  (lat: 599, lon: 706)
Coordinates:
  * lat      (lat) int32 2kB 0 1 2 3 4 5 6 7 ... 591 592 593 594 595 596 597 598
  * lon      (lon) int32 3kB 0 1 2 3 4 5 6 7 ... 698 699 700 701 702 703 704 705
Data variables:
    *empty*